## ASSIGNMENT WEEK 2

In [1]:
import pandas as pd
df = pd.read_csv('Online Retail-2.csv', encoding = "ISO-8859-1")
s2 =df.groupby('InvoiceNo')['StockCode'].apply(','.join)
s2 = s2.reset_index()
s2[:5].to_csv('Online Retail-2.txt', columns = ["StockCode"], header=False, index=False)

In [3]:
from pyspark.mllib.fpm import FPGrowth
from pyspark import SparkContext
sc = SparkContext()

In [4]:
data = sc.textFile("Online Retail-2.txt")
transactions = data.map(lambda line: line.strip().split(' '))

In [5]:
model = FPGrowth.train(transactions, minSupport = 0.15, numPartitions = 10)
freqItemsets = model.freqItemsets().collect()

freqItemsetsDict = {}

for i in range(0, len(freqItemsets)):
    freqItemsetsDict[str(freqItemsets[i].items)] = freqItemsets[i].freq

### 1) How many frequent itemsets were found with minimum support of .15?

In [6]:
freqItemsets

[FreqItemset(items=['21756'], freq=1),
 FreqItemset(items=['"22633,22632"'], freq=1),
 FreqItemset(items=['"22960,22913,22912,22914"'], freq=1),
 FreqItemset(items=['"85123A,71053,84406B,84029G,84029E,22752,21730"'], freq=1),
 FreqItemset(items=['"84879,22745,22748,22749,22310,84969,22623,22622,21754,21755,21777,48187"'], freq=1)]

### 2) How many association rules were identified with a minimum confidence of .4?

In [7]:
minConfidence = .4
rules = sorted(model._java_model.generateAssociationRules(minConfidence).collect(), 
    key=lambda x: x.confidence(), reverse=True)

for j in range(0, len(rules)):
    antecedent = list(rules[j].antecedent())
    consequent = list(rules[j].consequent())
    ruleItemset = str(antecedent + consequent)
    
    try:
        support = "{0:.15f}".format(freqItemsetsDict[ruleItemset] / transactions.count())
    except KeyError:
        next
    
    confidence = "{0:.15f}".format(rules[j].confidence())
    
    try:
        print(antecedent, '=>', consequent, '(support: '+ str(support), 'confidence: '+ str(confidence) + ')')
    except NameError:
        next

##### we have just only 5 transaction in online retail-2.txt so with a minimum confidence of .4 we don't have any association rules.

### 3) What is the support and confidence of the following association rule: ['21754', '22748'] => ['22745']? Go beyond reporting the values, but include an interpretation of these evaluation metrics.

#### the minsupport=0.15 so the associatin rule would have support between .15 and 1.
#### the minconfidence=0.4 so the associatin rule would have confidence between .4 and 1.